# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 1 2022 9:21AM,245502,12,HH-36684,Hush Hush,Released
1,Sep 1 2022 9:21AM,245502,12,HH-36685,Hush Hush,Released
2,Sep 1 2022 9:21AM,245502,12,HH-36686,Hush Hush,Released
3,Sep 1 2022 9:21AM,245502,12,HH-36687,Hush Hush,Released
4,Sep 1 2022 9:21AM,245502,12,HH-36688,Hush Hush,Released
5,Sep 1 2022 9:21AM,245502,12,HH-36689,Hush Hush,Released
6,Sep 1 2022 9:21AM,245502,12,HH-36690,Hush Hush,Released
7,Sep 1 2022 9:21AM,245502,12,HH-36691,Hush Hush,Released
8,Sep 1 2022 9:21AM,245502,12,HH-36692,Hush Hush,Released
9,Sep 1 2022 9:21AM,245502,12,HH-36693,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,245498,Released,2
22,245499,Released,8
23,245500,Released,1
24,245501,Released,1
25,245502,Released,27


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245498,NaN,NaN,2.0
245499,NaN,NaN,8.0
245500,NaN,NaN,1.0
245501,NaN,NaN,1.0
245502,NaN,NaN,27.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,14.0,8.0,0.0
245415,0.0,1.0,0.0
245419,1.0,0.0,29.0
245470,0.0,0.0,1.0
245474,0.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,14,8,0
245415,0,1,0
245419,1,0,29
245470,0,0,1
245474,0,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,14,8,0
1,245415,0,1,0
2,245419,1,0,29
3,245470,0,0,1
4,245474,0,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,14,8,
1,245415,,1,
2,245419,1,,29
3,245470,,,1
4,245474,,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 1 2022 9:21AM,245502,12,Hush Hush
27,Sep 1 2022 9:14AM,245501,10,Emerginnova
28,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC."
29,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc."
37,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation
54,Sep 1 2022 9:03AM,245498,10,Beach Patient Assistance
55,Sep 1 2022 9:03AM,245498,10,Beach Products Inc
56,Sep 1 2022 8:59AM,245495,10,ISDIN Corporation
90,Sep 1 2022 8:58AM,245496,10,ISDIN Corporation
99,Sep 1 2022 8:57AM,245492,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 1 2022 9:21AM,245502,12,Hush Hush,,,27
1,Sep 1 2022 9:14AM,245501,10,Emerginnova,,,1
2,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",,,1
3,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",,,8
4,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,,,17
5,Sep 1 2022 9:03AM,245498,10,Beach Patient Assistance,,,2
6,Sep 1 2022 9:03AM,245498,10,Beach Products Inc,,,2
7,Sep 1 2022 8:59AM,245495,10,ISDIN Corporation,,,34
8,Sep 1 2022 8:58AM,245496,10,ISDIN Corporation,,,9
9,Sep 1 2022 8:57AM,245492,10,ISDIN Corporation,,,49


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 9:21AM,245502,12,Hush Hush,27,,
1,Sep 1 2022 9:14AM,245501,10,Emerginnova,1,,
2,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",1,,
3,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",8,,
4,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,17,,
5,Sep 1 2022 9:03AM,245498,10,Beach Patient Assistance,2,,
6,Sep 1 2022 9:03AM,245498,10,Beach Products Inc,2,,
7,Sep 1 2022 8:59AM,245495,10,ISDIN Corporation,34,,
8,Sep 1 2022 8:58AM,245496,10,ISDIN Corporation,9,,
9,Sep 1 2022 8:57AM,245492,10,ISDIN Corporation,49,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 9:21AM,245502,12,Hush Hush,27,,
1,Sep 1 2022 9:14AM,245501,10,Emerginnova,1,,
2,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",1,,
3,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",8,,
4,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 9:21AM,245502,12,Hush Hush,27.0,NaN,NaN
1,Sep 1 2022 9:14AM,245501,10,Emerginnova,1.0,NaN,NaN
2,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",1.0,NaN,NaN
3,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",8.0,NaN,NaN
4,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 9:21AM,245502,12,Hush Hush,27.0,0.0,0.0
1,Sep 1 2022 9:14AM,245501,10,Emerginnova,1.0,0.0,0.0
2,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",1.0,0.0,0.0
3,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",8.0,0.0,0.0
4,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3682375,143.0,1.0,0.0
12,490979,27.0,1.0,0.0
15,981861,59.0,0.0,1.0
19,490978,1.0,5.0,0.0
20,245403,0.0,8.0,14.0
21,245415,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3682375,143.0,1.0,0.0
1,12,490979,27.0,1.0,0.0
2,15,981861,59.0,0.0,1.0
3,19,490978,1.0,5.0,0.0
4,20,245403,0.0,8.0,14.0
5,21,245415,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,143.0,1.0,0.0
1,12,27.0,1.0,0.0
2,15,59.0,0.0,1.0
3,19,1.0,5.0,0.0
4,20,0.0,8.0,14.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,143.0
1,12,Released,27.0
2,15,Released,59.0
3,19,Released,1.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,1.0,0.0,14.0,0.0
Executing,1.0,1.0,0.0,5.0,8.0,1.0
Released,143.0,27.0,59.0,1.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,1.0,0.0,14.0,0.0
1,Executing,1.0,1.0,0.0,5.0,8.0,1.0
2,Released,143.0,27.0,59.0,1.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,1.0,0.0,14.0,0.0
1,Executing,1.0,1.0,0.0,5.0,8.0,1.0
2,Released,143.0,27.0,59.0,1.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()